In [0]:
import numpy as np

In [0]:
#starting experiment with convolution layer.
def basic_conv(layer,kernel):
  layer_size=layer.shape[0]
  kernel_size=kernel.shape[0]
  sub_start=0
  sub_end=layer_size-kernel_size+1
  output=[]
  for sub_row_start in range(sub_start,sub_end):
    sub_row_end=sub_row_start+kernel_size
    temp_row=[]
    for sub_col_start in range(sub_start,sub_end):
      sub_col_end=sub_col_start+kernel_size
      temp_mat=layer[sub_row_start:sub_row_end,sub_col_start:sub_col_end]*kernel
      temp_mat=np.sum(temp_mat)
      temp_row.append(temp_mat)
    output.append(temp_row)
  return np.array(output)

In [0]:
layer=[[1,1,1,2,1],[2,3,2,6,2],[4,5,6,3,3],[1,1,1,1,1],[3,3,3,4,5]]
kernel=[[1,1],[1,2]]
layer=np.array(layer)
kernel=np.array(kernel)

In [0]:
res=basic_conv(layer,kernel)

In [105]:
print(layer)
print(kernel)
print(res)

[[1 1 1 2 1]
 [2 3 2 6 2]
 [4 5 6 3 3]
 [1 1 1 1 1]
 [3 3 3 4 5]]
[[1 1]
 [1 2]]
[[10  9 17 13]
 [19 22 20 17]
 [12 14 12  9]
 [11 11 13 16]]


this is working perfectly fine now we have to archive the same with the help of Fast forier transform. but its good to verifly with a standard library first.

In [0]:
from scipy.ndimage import convolve
from scipy.ndimage import correlate
abs_res=convolve(layer, kernel,mode='wrap')
abs_res2=correlate(layer, kernel,mode='wrap')

In [0]:
from scipy import signal
abs_res3 = signal.correlate(layer,kernel, mode='valid')

In [108]:
print(abs_res)
print(abs_res2)
print(abs_res3)

[[ 8  8 12 13  7]
 [16 19 19 20 13]
 [15 18 17 11 12]
 [ 9  9 10 12 11]
 [11 11 13 16 15]]
[[11  9  9 12 13]
 [ 8 10  9 17 13]
 [15 19 22 20 17]
 [10 12 14 12  9]
 [13 11 11 13 16]]
[[10  9 17 13]
 [19 22 20 17]
 [12 14 12  9]
 [11 11 13 16]]


ok so the convolution aka corelation that we are doing are correct lets move forward.

we are able to get some partial information this is lets try to figure out things by doing further experiments.

i.e things are going ok with the wrap mode.

In [0]:
#starting with forer shit.
f_layer=np.fft.fft2(layer,(5,5))
f_kernel=np.fft.fft2(np.flip(kernel),(5,5))
f_mul=f_layer*f_kernel
r_mul=np.fft.ifft2(f_mul)

In [110]:
print(f_layer)

[[ 65.         +0.j          -4.73606798 +0.81229924j
   -0.26393202 -3.4409548j   -0.26393202 +3.4409548j
   -4.73606798 -0.81229924j]
 [ -4.83688104 -6.55139449j  -4.30901699 +7.83296612j
   -3.61803399 -5.42882455j   3.19098301 +1.76335576j
    2.07294902 -1.67759904j]
 [-12.66311896+16.98026002j   4.30901699 -2.85316955j
   -3.19098301 +2.21238373j   5.42705098 -3.66546879j
   -1.38196601 +4.53076859j]
 [-12.66311896-16.98026002j  -1.38196601 -4.53076859j
    5.42705098 +3.66546879j  -3.19098301 -2.21238373j
    4.30901699 +2.85316955j]
 [ -4.83688104 +6.55139449j   2.07294902 +1.67759904j
    3.19098301 -1.76335576j  -3.61803399 +5.42882455j
   -4.30901699 -7.83296612j]]


In [111]:

print(r_mul)
print(np.flip(kernel))

[[15.+0.j 11.+0.j 11.+0.j 13.+0.j 16.+0.j]
 [ 7.+0.j  8.+0.j  8.+0.j 12.+0.j 13.+0.j]
 [13.+0.j 16.+0.j 19.+0.j 19.+0.j 20.+0.j]
 [12.+0.j 15.+0.j 18.+0.j 17.+0.j 11.+0.j]
 [11.+0.j  9.+0.j  9.+0.j 10.+0.j 12.+0.j]]
[[2 1]
 [1 1]]


now we need to find how the padding and trucating is happending under the hood.

-> my assumption are somewat correct atleast about the padding.


In [117]:
kernel_list=np.flip(kernel).tolist()
kernel_size=kernel.shape[0]
for i in range(kernel_size):
  for j in range(kernel_size,5):
    kernel_list[i].append(0)
for i in range(kernel_size,5):
  kernel_list.append([])
  for j in range(5):
    kernel_list[i].append(0)

print(kernel_list)

[[2, 1, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [118]:
new_kernel=np.array(kernel_list)
print(new_kernel)

[[2 1 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [119]:
print(np.fft.fft2(new_kernel))
print('this is the second one')
print(np.fft.fft2(kernel,(5,5)))

[[5.        +0.00000000e+00j 3.61803399-1.90211303e+00j
  1.38196601-1.17557050e+00j 1.38196601+1.17557050e+00j
  3.61803399+1.90211303e+00j]
 [3.61803399-1.90211303e+00j 1.80901699-2.48989828e+00j
  0.69098301-9.51056516e-01j 1.80901699+5.87785252e-01j
  3.61803399+0.00000000e+00j]
 [1.38196601-1.17557050e+00j 0.69098301-9.51056516e-01j
  0.69098301-2.24513988e-01j 1.38196601-1.38777878e-17j
  1.80901699-5.87785252e-01j]
 [1.38196601+1.17557050e+00j 1.80901699+5.87785252e-01j
  1.38196601+1.38777878e-17j 0.69098301+2.24513988e-01j
  0.69098301+9.51056516e-01j]
 [3.61803399+1.90211303e+00j 3.61803399+0.00000000e+00j
  1.80901699-5.87785252e-01j 0.69098301+9.51056516e-01j
  1.80901699+2.48989828e+00j]]
this is the second one
[[ 5.00000000e+00+0.00000000e+00j  2.92705098e+00-2.85316955e+00j
  -4.27050983e-01-1.76335576e+00j -4.27050983e-01+1.76335576e+00j
   2.92705098e+00+2.85316955e+00j]
 [ 2.92705098e+00-2.85316955e+00j  2.22044605e-16-3.07768354e+00j
  -1.11803399e+00-3.63271264e-01j

In [0]:
#now we will do the convolution operation.
#new_kernel=np.flip(new_kernel)
temp_layer=np.fft.fft2(layer,(5,5))
temp_kernel=np.fft.fft2(new_kernel)
temp=temp_layer*temp_kernel
tt=np.fft.ifft2(temp)

In [121]:
print(tt)
print(temp_layer)

[[11.+0.j  9.+0.j  9.+0.j 12.+0.j 13.+0.j]
 [ 8.+0.j 10.+0.j  9.+0.j 17.+0.j 13.+0.j]
 [15.+0.j 19.+0.j 22.+0.j 20.+0.j 17.+0.j]
 [10.+0.j 12.+0.j 14.+0.j 12.+0.j  9.+0.j]
 [13.+0.j 11.+0.j 11.+0.j 13.+0.j 16.+0.j]]
[[ 65.         +0.j          -4.73606798 +0.81229924j
   -0.26393202 -3.4409548j   -0.26393202 +3.4409548j
   -4.73606798 -0.81229924j]
 [ -4.83688104 -6.55139449j  -4.30901699 +7.83296612j
   -3.61803399 -5.42882455j   3.19098301 +1.76335576j
    2.07294902 -1.67759904j]
 [-12.66311896+16.98026002j   4.30901699 -2.85316955j
   -3.19098301 +2.21238373j   5.42705098 -3.66546879j
   -1.38196601 +4.53076859j]
 [-12.66311896-16.98026002j  -1.38196601 -4.53076859j
    5.42705098 +3.66546879j  -3.19098301 -2.21238373j
    4.30901699 +2.85316955j]
 [ -4.83688104 +6.55139449j   2.07294902 +1.67759904j
    3.19098301 -1.76335576j  -3.61803399 +5.42882455j
   -4.30901699 -7.83296612j]]
